<a href="https://colab.research.google.com/github/westefns-souza/streamlit-example/blob/master/Trabalho_analise_redes_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analise da base de dados do twitter

In [ ]:
!pip install pyvis

In [ ]:
import re
import pandas as pd
import networkx as nx
from pyvis.network import Network
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import csv

# Limpando a base
Como a base não tinha um csv bom, tive que limpar principalmente a coluna 'friends' que estava como um array separado por vírgula, com isso a leitura quebrava no momento de abrir e ler os dados com o pandas. 

Achei Uma função de substituir_virgulas usando regex para identicar apenas as frases dentro de um array [ ]. 

In [ ]:
def substituir_virgulas(texto):
    padrao = r'\[(.*?)\]'  # padrão para encontrar o texto entre colchetes
    matches = re.findall(padrao, texto)  # encontrar todas as ocorrências entre colchetes
    for match in matches:
        novo_texto = match.replace(',', ';')  # substituir vírgulas por ponto-e-vírgula
        texto = texto.replace(match, novo_texto)  # substituir a ocorrência original pelo novo texto
    return texto

## Criando um novo arquivo já corrigido

Para que não precise trabalhar com os dados em memória sempre

In [ ]:
with open("/content/data_twitter.csv") as file:
  with open("/content/data_twitter_formated.csv", "w") as new_file:
      for line in file:
        new_file.write(substituir_virgulas(line))

Carregando os dados do arquivo formatado para um DateFrame

In [ ]:
df = pd.read_csv('/content/data_twitter_formated.csv', on_bad_lines="skip")
df

,id,screenName,tags,avatar,followersCount,friendsCount,lang,lastSeen,tweetId,friends
0,1969527638,LlngoMakeEmCum_,"[ ""#nationaldogday"" ]",http://pbs.twimg.com/profile_images/5342862178...,319,112,en,1472271687519,769310701580083200,"[ ""1969574754""; ""1969295556""; ""1969284056""; ""1..."
1,51878493,_notmichelle,"[ ""#nationaldogday"" ]",http://pbs.twimg.com/profile_images/7619776021...,275,115,en,1472270622663,769309490038439936,"[ ""60789485""; ""2420931980""; ""2899776756""; ""127..."
2,1393409100,jesseayye,"[ ""#narcos"" ]",http://pbs.twimg.com/profile_images/7132829382...,120,107,en,1472804144409,771622623583186944,"[ ""86868062""; ""19697415""; ""2998836604""; ""45629..."
3,232891415,MrBrianLloyd,"[ ""#gloryoutnow"" ]",http://pbs.twimg.com/profile_images/1334406683...,492,325,en,1472269186776,769308131696467969,"[ ""361335082""; ""1405248468""; ""24626354""; ""7256..."
4,710130422907207680,sarahdorat_16,"[ ""#nationaldogday"" ]",http://pbs.twimg.com/profile_images/7671805202...,128,218,en,1472271397356,769309769144283137,"[ ""1571896093""; ""768938323612008448""; ""2548665..."
...,...,...,...,...,...,...,...,...,...,...
10223,3191601721,LuffBri,"[ ""#respecttylerjoseph"" ]",http://pbs.twimg.com/profile_images/7686781088...,197,428,en,1472258135177,769297456156647424,"[ ""3410690243""; ""1319599614""; ""391013063""; ""33..."
10224,1390557145,giselleali_,"[ ""#nationaldogday"" ]",http://pbs.twimg.com/profile_images/7450518741...,252,213,en,1472257000572,769297455670329344,"[ ""21526685""; ""760567715979464704""; ""722898489..."
10225,1908002540,DukesBoots,"[ ""#nationaldogday"" ]",http://pbs.twimg.com/profile_images/5807645934...,1070,1984,en,1472257754077,769297455506661376,"[ ""154564075""; ""114774201""; ""338299807""; ""7343..."
10226,31087443,aniyah2105,"[ ""#nationaldogday"" ]",http://pbs.twimg.com/profile_images/6214343401...,1127,2369,en,1472257002155,769297457159151616,"[ ""607042548""; ""14942684""; ""768509276130127872..."


Selecionando apenas as linhas que estou utilizando

In [ ]:
df_reduced = df[['id', 'friends']]
df_reduced

,id,friends
0,1969527638,"[ ""1969574754""; ""1969295556""; ""1969284056""; ""1..."
1,51878493,"[ ""60789485""; ""2420931980""; ""2899776756""; ""127..."
2,1393409100,"[ ""86868062""; ""19697415""; ""2998836604""; ""45629..."
3,232891415,"[ ""361335082""; ""1405248468""; ""24626354""; ""7256..."
4,710130422907207680,"[ ""1571896093""; ""768938323612008448""; ""2548665..."
...,...,...
10223,3191601721,"[ ""3410690243""; ""1319599614""; ""391013063""; ""33..."
10224,1390557145,"[ ""21526685""; ""760567715979464704""; ""722898489..."
10225,1908002540,"[ ""154564075""; ""114774201""; ""338299807""; ""7343..."
10226,31087443,"[ ""607042548""; ""14942684""; ""768509276130127872..."


Separando as edges juntamente com os nós, foi necessário porque o coluna de friends se torna uma lista

In [ ]:
nodes = []
edges = []

In [ ]:
for line in df_reduced.values.tolist():
  userId = int(line[0])
  nodes.append(userId)

  friends = line[1].replace('[', '').replace('; ]', '').replace(']', '').replace('"', '').split(';')

  for friend in friends:
    friend = friend.strip()
    if friend != '':
      friend = int(friend)
      nodes.append(friend)
      edges.append([userId, friend])

Criando um arquvivo apenas com os nós para usar depois sem precisar dos passos acima

In [ ]:
import csv

with open('base_reduzida.csv', 'w', newline='') as csvfile:
    fieldnames = ['userId', 'friendId']
    writer = csv.DictWriter(csvfile, fieldnames)
    
    writer.writeheader()
    for linha in edges:
      writer.writerow({'userId': linha[0], 'friendId': linha[1]})

# Usando apartir da base reduzida para criar od dados

In [ ]:
df_base_reduzida = pd.read_csv('/content/base_reduzida.csv', on_bad_lines="skip")

## Criando o Grafico com NetworkX

In [ ]:
G = nx.Graph()

for line in df_base_reduzida.values.tolist():
  G.add_nodes_from(line)
  G.add_edges_from([(line[0], line[1])])

## Gerando o gráfico com o Pyvis

In [ ]:
nt = Network('500px', '100%', notebook=True, cdn_resources='in_line')
nt.from_nx(G)
nt.show('grafico.html')
display(HTML('grafico.html'))

# Obtendo os dados pedidos na máteria

## Obtendo a matriz de adjacência

In [ ]:
adj_matrix = nx.to_pandas_adjacency(G)

plt.imshow(adj_matrix, cmap='binary', origin='upper')
plt.xlabel('Nós')
plt.ylabel('Nós')
plt.title('Matriz de Adjacência')
plt.colorbar()
plt.show()

## Calcular o diâmetro

In [ ]:
diameter = nx.diameter(G)
print("Diâmetro: ", diameter)

## Calcular a periferia

In [ ]:
periphery = nx.periphery(G)

print("Periferia: ", periphery)

 ## Histograma de distribuição empírica de grau.

In [ ]:
hist = nx.degree_histogram(G)
degrees = list(range(len(hist)))
frequencies = hist

nt = Network(height="500px", width="100%", notebook=True,cdn_resources='in_line')
nt.from_nx(G)

for degree, freq in zip(degrees, frequencies):
    nt.add_node(degree, size=freq)
  
nt.show("histograma.html")
display(HTML('histograma.html'))

## Coeficiente de clustering local para nós escolhidos.



In [ ]:
coefs_clustering = nx.clustering(G, [1969527638])
coefs_clustering

## Calcular o coeficiente de clustering global

In [ ]:
coef_clustering = nx.average_clustering(G)
coef_clustering

## Encontrar os componentes conectados fortemente

In [ ]:
componentes_fortemente = nx.strongly_connected_components(G)
componentes_fortemente

## Encontrar os componentes conectados fracamente

In [ ]:
componentes_fracamente = nx.weakly_connected_components(G)
componentes_fracamente

## Calcular a centralidade de grau

In [ ]:
centrality = nx.degree_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Grau: ", centrality_value)

## Calcular a centralidade de proximidade

In [ ]:
centrality = nx.closeness_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Proximidade: ", centrality_value)

## Calcular o Betweenness centrality

In [ ]:
centrality = nx.betweenness_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Intermediação: ", centrality_value)

## Calcular a centralidade de eigenvector

In [ ]:
centrality = nx.eigenvector_centrality(G)

for node, centrality_value in centrality.items():
    print("Nó:", node, "Centralidade de Eigenvector: ", centrality_value)

## Calcular a assortatividade geral

In [ ]:
assortativity = nx.degree_assortativity_coefficient(G)
assortativity